<a href="https://colab.research.google.com/github/GitMarco27/TMML/blob/main/Notebooks/010_Lambda_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3 Minutes Machine Learning
## Episode 10: Lambda Layer

#### Marco Sanguineti, 2021
---
Welcome to 3 minutes Machine Learning!

In [1]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

In [3]:
import os
def loadThumb(path):
  # Let's import this video thumbnail!
  if os.path.exists(path):
    myThumb = plt.imread(path)
    fig, ax = plt.subplots(figsize=(15, 10))
    plt.axis('off')
    ax.imshow(myThumb)
    plt.show()

loadThumb('/tmp/yt_thumb_010.png')


#### Video Topics
> 1. Load the fashion MNIST dataset
> 2. Create a model with the keras API with a lambda layer for normalization
> 3. Train the model and check the results
> 4. See you on next video! 

# Load the dataset
___

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
class_names

In [6]:
def plot_my_image(image):
  plt.figure()
  plt.imshow(image)
  plt.colorbar()
  plt.grid(False)
  plt.show()

def plot_images(images):
  plt.figure(figsize=(10,10))
  for i in range(25):
      plt.subplot(5,5,i+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      plt.imshow(images[i], cmap=plt.cm.binary)
      plt.xlabel(class_names[train_labels[i]])
  plt.show()

In [ ]:
plot_my_image(train_images[22])

In [ ]:
plot_images(train_images)

# Create the model
___

In [9]:
from tensorflow.keras.layers import Dense, Input, Lambda, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer

In [ ]:
print(f'Images Shape: {train_images[0].shape}')
print(f'Num. of classes: {len(set(train_labels))}')

In [ ]:
input_data = Input(shape=(train_images[0].shape), name='Input')

normalization = Lambda(lambda x: x / 255.0)(input_data)

flatten = Flatten()(normalization)
dense = Dense(64, activation='relu')(flatten)
output = Dense(len(set(train_labels)), name='output', activation='softmax')(dense)
model = Model(input_data, output)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

# Train and check the results
___

In [ ]:
model.layers

In [ ]:
history = model.fit(
    train_images,
    train_labels,
    batch_size = 256,
    epochs=40,
    validation_data=(test_images,
                     test_labels))

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
fig, ax = plt.subplots(figsize=(8, 6))
plt.plot(loss, label='loss')
plt.plot(val_loss, label = 'val loss')
plt.plot(accuracy, label='accuracy')
plt.plot(val_accuracy, label='val accuracy')
plt.grid('both')
plt.xlabel('Epochs')
plt.ylabel('Loss Function')
plt.title('Loss Function trend')
plt.legend()
plt.show()

In [ ]:
input_data = Input(shape=(*train_images[0].shape,), name='Input')

normalization = Lambda(lambda x: x / 255.0)(input_data)

expand = Lambda(lambda x: tf.expand_dims(x, -1))(normalization)

conv_1 = Conv2D(64, (3, 3), activation='relu')(expand)
max_pool_1 = MaxPool2D(2, 2)(conv_1)
conv_2 = Conv2D(64, (3, 3), activation='relu')(max_pool_1)
max_pool_2 = MaxPool2D(2, 2)(conv_2)
flatten = Flatten()(max_pool_2)
output = Dense(len(set(train_labels)), name='output', activation='softmax')(flatten)
model = Model(input_data, output)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=True,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96
)

In [ ]:
history = model.fit(
    train_images,
    train_labels,
    batch_size = 256,
    epochs=40,
    validation_data=(test_images,
                     test_labels))

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
fig, ax = plt.subplots(figsize=(8, 6))
plt.plot(loss, label='loss')
plt.plot(val_loss, label = 'val loss')
plt.plot(accuracy, label='accuracy')
plt.plot(val_accuracy, label='val accuracy')
plt.grid('both')
plt.xlabel('Epochs')
plt.ylabel('Loss Function')
plt.title('Loss Function trend')
plt.legend()
plt.show()


In [ ]:
model.predict((tf.expand_dims(train_images[0], axis=0)))

In [21]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
import numpy as np
predictions = model.predict(test_images)
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

# Greetings
---

In [ ]:
!pip install art
from art import tprint, aprint
tprint('See you on next videos!')
def subscribe():
  """
  Attractive subscription form
  """
  aprint("giveme", number=5)
  print(f'\n\tLike and subscribe to support this work!\n')
  aprint("giveme", number=5)
subscribe()